In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas import Timedelta, Timestamp
from collections import namedtuple

from IPython.core.debugger import set_trace

In [2]:
df_estacao = pd.read_csv("DATA/dados_estacoes_5min.csv",
        parse_dates=["DATE"],
        index_col=["DATE"])

df_radar = pd.read_csv("DATA/dados_radar_semHorarioVerao.csv",
                      parse_dates=['DATE'],
                      index_col=["DATE"])

df_estacao.sort_index(inplace=True)
df_radar.sort_index(inplace=True)

In [14]:
df_est = df_estacao[df_estacao > 0].dropna(how="all",axis=0).dropna(how="all",axis=1)
indices = df_est.index
eventos = []

evento = []
last_indice = indices[0]
evento.append(last_indice)
for indice in indices[1:]:
    evento.append(indice)
    # Eventos com um gap maior que 10 min são eventos distintos
    if indice - last_indice > Timedelta("10min"):
        # eventos maiores que 10 min
        if len(evento) > 1:
            eventos.append(evento)
        evento = []
    last_indice = indice
print(len(eventos))
    

1690


In [15]:
# Evento aleátorio para tests
evento = eventos[3]
evento

[Timestamp('2016-01-01 15:50:00'),
 Timestamp('2016-01-01 16:00:00'),
 Timestamp('2016-01-01 16:10:00'),
 Timestamp('2016-01-01 16:20:00'),
 Timestamp('2016-01-01 16:30:00'),
 Timestamp('2016-01-01 16:40:00'),
 Timestamp('2016-01-01 16:50:00'),
 Timestamp('2016-01-01 19:00:00')]

In [16]:
df_est.loc[evento].dropna(axis=1, how="all")

,MB_GAM1,MB_PRO1,MB_PRO2,MB_SEG1
DATE,,,,
2016-01-01 15:50:00,NaN,NaN,NaN,15.50
2016-01-01 16:00:00,NaN,3.00,NaN,2.50
2016-01-01 16:10:00,NaN,9.75,6.75,NaN
2016-01-01 16:20:00,NaN,5.50,5.75,NaN
2016-01-01 16:30:00,NaN,1.00,4.75,NaN
2016-01-01 16:40:00,0.25,NaN,NaN,0.25
2016-01-01 16:50:00,0.50,NaN,NaN,NaN
2016-01-01 19:00:00,NaN,1.75,NaN,NaN


In [17]:
df_evento = df_est.loc[evento].dropna(axis=1, how="all")
evento_estacao = df_evento.MB_PRO1.dropna()
evento_estacao

DATE
2016-01-01 16:00:00    3.00
2016-01-01 16:10:00    9.75
2016-01-01 16:20:00    5.50
2016-01-01 16:30:00    1.00
2016-01-01 19:00:00    1.75
Name: MB_PRO1, dtype: float64

In [18]:
evento_estacao.index

DatetimeIndex(['2016-01-01 16:00:00', '2016-01-01 16:10:00',
               '2016-01-01 16:20:00', '2016-01-01 16:30:00',
               '2016-01-01 19:00:00'],
              dtype='datetime64[ns]', name='DATE', freq=None)

É necessario ainda separar os eventos por estação, como é possivel ver acima os eventos começam em horarios distintos por estaçoes.


In [19]:
df = pd.DataFrame(columns=["start", "station","duration","Pluviometro","RADAR"])
stations = df_est.columns
for station in stations:
    df_station = df_est[station]
    df_station_nonull = df_station[df_station > 0].dropna()
    indices = df_station_nonull.index
    dict_evento = {}
    evento = []
    last_indice = indices[0]
    evento.append(last_indice)
    for indice in indices[1:]:
        # Eventos com um gap maior que 10 min são eventos distintos
        if indice - last_indice > Timedelta("10min"):
            # eventos maiores que 10 min
            if len(evento) > 1:
                dict_evento["start"] = evento[0]
                dict_evento["station"] = station
                dict_evento["duration"] = evento[-1] - evento[0]
                dict_evento["Pluviometro"] = df_station.loc[evento].sum()
                try:
                    dict_evento["RADAR"] = df_radar[station].loc[evento].sum()
                except KeyError:
                    dict_evento["RADAR"] = np.nan
                
                df = df.append(dict_evento, ignore_index=True)
            evento = []
        else:
            evento.append(indice)
        last_indice = indice

    

/home/rossuna/anaconda3/envs/ic/lib/python3.6/site-packages/ipykernel_launcher.py:21: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike


1690


In [21]:
df.head()

,start,station,duration,Pluviometro,RADAR
0,2016-10-14 08:20:00,MB_ANHA1,00:30:00,3.00,1.505884
1,2016-10-14 09:20:00,MB_ANHA1,01:20:00,4.25,2.097585
2,2016-10-14 11:10:00,MB_ANHA1,00:10:00,1.50,NaN
3,2016-10-20 16:10:00,MB_ANHA1,00:30:00,9.75,0.880495
4,2016-10-20 19:10:00,MB_ANHA1,01:30:00,8.00,2.602351


In [25]:
df["ERROR"] = (df.Pluviometro - df.RADAR)

In [26]:
df.head()

,start,station,duration,Pluviometro,RADAR,ERROR
0,2016-10-14 08:20:00,MB_ANHA1,00:30:00,3.00,1.505884,1.494116
1,2016-10-14 09:20:00,MB_ANHA1,01:20:00,4.25,2.097585,2.152415
2,2016-10-14 11:10:00,MB_ANHA1,00:10:00,1.50,NaN,NaN
3,2016-10-20 16:10:00,MB_ANHA1,00:30:00,9.75,0.880495,8.869505
4,2016-10-20 19:10:00,MB_ANHA1,01:30:00,8.00,2.602351,5.397649


In [27]:
df.ERROR.describe()

count    4410.000000
mean        5.643678
std       137.681463
min       -53.774293
25%         0.125000
50%         0.825369
75%         3.006450
max      8992.978052
Name: ERROR, dtype: float64

In [ ]:
df.to_csv("DATA/eventos.csv")